In [1]:
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer        

In [2]:
text_pos = []
labels_pos = []
with open("./pos_tweets.txt") as f:
    for i in f: 
        text_pos.append(i) 
        labels_pos.append('pos')

text_neg = []
labels_neg = []
with open("./neg_tweets.txt") as f:
    for i in f: 
        text_neg.append(i)
        labels_neg.append('neg')

In [3]:
training_text = text_pos[:int((.8)*len(text_pos))] + text_neg[:int((.8)*len(text_neg))]
training_labels = labels_pos[:int((.8)*len(labels_pos))] + labels_neg[:int((.8)*len(labels_neg))]

test_text = text_pos[int((.8)*len(text_pos)):] + text_neg[int((.8)*len(text_neg)):]
test_labels = labels_pos[int((.8)*len(labels_pos)):] + labels_neg[int((.8)*len(labels_neg)):]

In [4]:
vectorizer = CountVectorizer(
    analyzer = 'word',
    lowercase = False,
    max_features = 85
)

In [5]:
features = vectorizer.fit_transform(
    training_text + test_text)

features_nd = features.toarray() # for easy use

In [6]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        features_nd[0:len(training_text)], 
        training_labels,
        train_size=0.80, 
        random_state=1234)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()

In [8]:
log_model = log_model.fit(X=X_train, y=y_train)

In [9]:
y_pred = log_model.predict(X_test)

In [10]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.7601246105919003


In [11]:
from sklearn.ensemble import RandomForestClassifier
rf_clf=RandomForestClassifier(random_state=0)
rf_clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [12]:
y_pred = rf_clf.predict(X_test)

In [13]:
print(accuracy_score(y_test, y_pred))

0.7414330218068536


In [14]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        neg       0.77      0.89      0.82       219
        pos       0.64      0.43      0.51       102

avg / total       0.73      0.74      0.73       321



In [15]:
print X_train.shape
print X_test.shape
print len(y_train)
print len(y_test)

(1281, 85)
(321, 85)
1281
321


In [16]:
#Artificial Neural Networks
from keras.models import Sequential
from keras.layers import *

model=Sequential()

model.add(Dense(32,activation='relu',input_dim=85))
model.add(Dense(64,activation='relu'))
#model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                2752      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 4,929
Trainable params: 4,929
Non-trainable params: 0
_________________________________________________________________


In [18]:
from sklearn.preprocessing import LabelEncoder
import keras
lb = LabelEncoder()
train_labels = lb.fit_transform(y_train)
test_labels=lb.fit_transform(y_test)
#train_labels = keras.utils.np_utils.to_categorical(train_labels)

In [19]:
model.fit(X_train,train_labels,batch_size=32,epochs=50,validation_data=(X_test,test_labels),verbose=1)

Train on 1281 samples, validate on 321 samples
Epoch 1/50
1281/1281 [==============================] - 0s 218us/step - loss: 0.6356 - acc: 0.6792 - val_loss: 0.6085 - val_acc: 0.6822
Epoch 2/50
1281/1281 [==============================] - 0s 118us/step - loss: 0.5905 - acc: 0.6948 - val_loss: 0.5882 - val_acc: 0.6822
Epoch 3/50
1281/1281 [==============================] - 0s 117us/step - loss: 0.5612 - acc: 0.6995 - val_loss: 0.5616 - val_acc: 0.7009
Epoch 4/50
1281/1281 [==============================] - 0s 108us/step - loss: 0.5301 - acc: 0.7237 - val_loss: 0.5350 - val_acc: 0.7259
Epoch 5/50
1281/1281 [==============================] - 0s 129us/step - loss: 0.4989 - acc: 0.7533 - val_loss: 0.5123 - val_acc: 0.7508
Epoch 6/50
1281/1281 [==============================] - 0s 109us/step - loss: 0.4705 - acc: 0.7681 - val_loss: 0.4963 - val_acc: 0.7695
Epoch 7/50
1281/1281 [==============================] - 0s 104us/step - loss: 0.4476 - acc: 0.7806 - val_loss: 0.4864 - val_acc: 0.7788
E

In [20]:
y_pred = model.predict_classes(X_test)
print(classification_report(y_pred, test_labels))


             precision    recall  f1-score   support

          0       0.82      0.79      0.81       227
          1       0.54      0.59      0.56        94

avg / total       0.74      0.73      0.74       321

